In [1]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import sys
import pandas as pd
import numpy as np
import csv
import random
import re
from sklearn.decomposition import TruncatedSVD
from torch.nn.utils.rnn import pad_sequence#用来填充序列
import time
from prefetch_generator import BackgroundGenerator
from torch.utils.tensorboard import SummaryWriter
from pywick.optimizers.nadam import Nadam#使用pywick包里的nadam优化器
from torch.optim import lr_scheduler

In [2]:
filepath = '/home/jsy/train_20160701-20170531/2016-10-02/876772.csv'

In [6]:
def tsvd(frame):
        tsvd = TruncatedSVD(1)
        if frame.shape[0] != 1:
            newframe = tsvd.fit_transform(np.transpose(frame))#降维成（1,10）的矩阵
        else:
            return frame.reshape((10,1))#第一行需要reshape一下
        return newframe
    
def mrx2vec(flist):#把截断奇异值的方法把矩阵变成向量(matrix2vec/img2vec)，传入：len(frametimelist)*(306*10),传出：len(frametimelist)*10
    vectensor = np.array(list(map(tsvd,flist))).squeeze(2)
    #veclist = veclist.transpose()
    #vectensor = torch.from_numpy(veclist)#转成张量
    return vectensor#传出一个形状为(1,序列长度,10)的张量，因为后面传入模型之前，还需要做一下pad_sequence(0维是batch_size维)


In [7]:
#原版
start = time.time()
data = pd.read_csv(filepath)#读取文件
data = data.drop(columns=['league','zhudui','kedui','companyname'])#去除非数字的列
frametimelist=data.frametime.value_counts().sort_index(ascending=False).index#将frametime的值读取成列表
framelist =list()#framelist为一个空列表,长度与frametimelist相同,一定要规定好具体形状和float类型，否则dataloader无法读取
'''
此处两个循环算法太慢，用pandas更慢，完全抛弃pandas后，数据处理速度从109秒降到了10秒，降到10秒后cpu利用率20%，再往上提也提不上去了，可能需要C++或C来写了
'''
new_data = np.array(data)
lables = new_data[:,0]
if len(frametimelist)>250:
    frametimelist = [frametimelist[0]]+random.sample(list(frametimelist)[1:-1],248)+[frametimelist[-1]]#如果长度大于500,保留头尾，并在中间随机抽取498个，共计500个
    frametimelist.sort(reverse=True)#并降序排列
for i in frametimelist:
    state = new_data[lables==i]#从第一次变盘开始得到当次转移
    #state = np.array(state)#不必转成numpy多维数组，因为已经是了
    state = np.delete(state,(0,1), axis=-1)#去掉frametime和cid
    #在填充成矩阵之前需要知道所有数据中到底有多少个cid
    framelist.append(state)
frametimelist = np.array(frametimelist)
vectensor = mrx2vec(framelist)
len_frame = vectensor.shape[0]
if len_frame<250:
    vectensor = np.concatenate((np.zeros((250-len_frame,10),dtype=np.float64),vectensor),axis=0)#如果不足500，则在前面用0填充
vectensor = torch.from_numpy(vectensor)
vectensor#传出一个帧列表,也可以把frametimelist一并传出来，此处暂不考虑位置参数的问题
end = time.time()
print('耗时'+str(end-start)+'秒')

耗时0.5449409484863281秒


In [18]:
vectensor.shape

torch.Size([250, 10])

In [4]:
#新版
start = time.time()
all_mean = np.array([0.92236896, 2.56799437, 3.64873151, 3.94965805])#全样本均值
all_std = np.array([0.03154385, 1.76454825, 1.0027637 , 3.01005411])#全样本标准差
data = pd.read_csv(filepath)#读取文件
data = data.drop(columns=['league','zhudui','kedui','companyname'])#去除非数字的列
frametimelist=data.frametime.value_counts().sort_index(ascending=False).index#将frametime的值读取成列表
framelist =list()#framelist为一个空列表,长度与frametimelist相同,一定要规定好具体形状和float类型，否则dataloader无法读取
'''
此处两个循环算法太慢，用pandas更慢，完全抛弃pandas后，数据处理速度从109秒降到了10秒，降到10秒后cpu利用率20%，再往上提也提不上去了，可能需要C++或C来写了
'''
new_data = np.array(data)
lables = new_data[:,0]
if len(frametimelist)>250:#如果总帧数大于250,则随机挑选250，若不足250,则在后面处理vectensor时补0
    frametimelist = [frametimelist[0]]+random.sample(list(frametimelist)[1:-1],248)+[frametimelist[-1]]#如果长度大于500,保留头尾，并在中间随机抽取498个，共计500个
    frametimelist.sort(reverse=True)#并降序排列
for i in frametimelist:
    state = new_data[lables==i]#从第一次变盘开始得到当次转移
    #state = np.array(state)#不必转成numpy多维数组，因为已经是了
    state = np.delete(state,(0,1), axis=-1)#去掉frametime和cid
    #在填充成矩阵之前需要知道所有数据中到底有多少个cid
    max = (state.max(axis=0)[0:4]-all_mean)/all_std#标准化
    mean = (state.mean(axis=0)[0:4]-all_mean)/all_std
    min = (state.min(axis=0)[0:4]-all_mean)/all_std
    max_min = np.concatenate((max,mean,min),axis=0)
    framelist.append(max_min)
frametimelist = np.array(frametimelist)
vectensor = np.array(framelist)
len_frame = vectensor.shape[0]
if len_frame<250:
    vectensor = np.concatenate((np.zeros((250-len_frame,12),dtype=np.float64),vectensor),axis=0)#如果不足500，则在前面用0填充
vectensor = torch.from_numpy(vectensor)
vectensor#传出一个帧列表,也可以把frametimelist一并传出来，此处暂不考虑位置参数的问题
end = time.time()
print('耗时'+str(end-start)+'秒')

耗时0.33396363258361816秒


In [5]:
vectensor

tensor([[-0.3921,  0.2448, -0.3478,  ...,  0.2448, -0.3478, -0.5979],
        [ 0.8760,  1.4916,  0.4002,  ...,  0.2448, -0.3478, -0.7740],
        [ 0.8760,  1.4916,  0.4002,  ...,  0.2448, -0.3478, -0.7740],
        ...,
        [ 1.8270,  1.9450,  0.6495,  ...,  0.2448, -0.4475, -0.8304],
        [ 1.8270,  1.9450,  0.6495,  ...,  0.2448, -0.4475, -0.8304],
        [ 1.8270,  1.9450,  0.6495,  ...,  0.2448, -0.4974, -0.8304]],
       dtype=torch.float64)

In [47]:
vectensor = np.array(framelist)

In [51]:
vectensor = np.delete(vectensor,[1,50],axis=0)

In [53]:
len_frame = vectensor.shape[0]
if len_frame<250:
    vectensor = np.concatenate((np.zeros((250-len_frame,12),dtype=np.float64),vectensor),axis=0)#如果不足500，则在前面用0填充
vectensor = torch.from_numpy(vectensor)
vectensor#传出一个帧列表,也可以把frametimelist一并传出来，此处暂不考虑位置参数的问题

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.9100, 3.0000, 3.3000,  ..., 3.0000, 3.3000, 2.1500],
        ...,
        [0.9800, 6.0000, 4.3000,  ..., 3.0000, 3.2000, 1.4500],
        [0.9800, 6.0000, 4.3000,  ..., 3.0000, 3.1500, 1.4500],
        [0.9800, 6.0000, 4.3000,  ..., 3.0000, 3.1500, 1.4500]],
       dtype=torch.float64)

In [54]:
vectensor.shape

torch.Size([250, 12])

In [45]:
frametimelist=data.frametime.value_counts().sort_index(ascending=False).index

In [46]:
len(frametimelist)

2038

In [37]:
np.array(framelist).shape

(250, 8)

In [19]:
a = framelist[249]

In [26]:
a.max(axis=0)[0:4]

array([0.98, 6.  , 4.3 , 2.15])

In [24]:
a.min(axis=0)

array([0.79  , 3.    , 3.15  , 1.45  , 0.1626, 0.224 , 0.4223, 0.64  ,
       0.77  , 0.78  ])

In [27]:
a.min(axis=0)[0:4]

array([0.79, 3.  , 3.15, 1.45])